In [1]:
import sys
import numpy
import math

print(sys.version_info)

sys.version_info(major=3, minor=8, micro=8, releaselevel='final', serial=0)


## Common Parameters and what they mean
G = specific heat ratio.  
RADB = OUTER RADIUS (in m, for calculations in real quantities)  
RADA = INNER RADIUS (in m,  ditto )  
Z0   = LENGTH OF ANODE (in m, ditto)   
C    = RADB/RADA.   
F    = Z0/RADA.   
L0   = CIRCUIT STRAY INDUCTANCE (in HENRY, ditto).  
C0   = ENERGY STORAGE CAPACITANCE (in FARAD, ditto).   
AL= capacitor time T0/TA, axial run-down time.  
BE= full axial phase inductance LZ0/L0.  
MASSF= REDUCED MASS FACTOR DUE TO MASS SHEDDING.   
CURRF= REDUCED CURRENT FACTOR DUE TO CURRENT SHEDDING.   
R0 IS STRAY CIRCUIT RESISTANCE IN OHM   
RESF=R0/Surge Impedance   
Z1 is end position, to end calculation of phase 5.  
MASSF, MASSFR are incorporated in TA & AL & in AA.  
CURRF IS INcorporated into BE.  
For Phase I calculations, T = Time, Z = Axial position (normalized to Zo).   
I = Current, ZZ = Speed, AC = Acceleration.  
II = Current derivative, I0 = Current Intergral, all normalized.   
HOWEVER ALL QUANTITIES WITH An R ATTACHED, HAVE BEEN RE-COMPUTED TO GIVE LABORATORY VALUES; e.g. TR IS TIME RE-COMPUTED IN MICROSEC;IR IS CURRENT RE-COMPUTED IN kA AND SO ON.  
D = Time increment, V = tube voltage, all normalized.   


In [2]:
ENDFLAG = 0
DFLAG = 0
DVFLAG = 0
NTN = 0
NBN = 0
NN = 0
VRMAX = 0
peakvs = 0
peakvp = 0
tc5 = 1
#For now, default to UNUICTPPFF device 
L0 = 110
C0 = 30
RADB = 3.2
RADA = 0.95
Z0 = 5
R0 = 12
V0 = 15
P0 = 3.5
MW = 4
#atomic z number
ZN = 1
dissociatenumber = 2
massf = 0.08
CURRF = 0.7
massfr = 0.16
currfr = 0.7

R0 = R0 / 1000
#Setting consts
#magnetic permitivity of vaccum, Henry per meter 
MU = 1.257e-6
# \pi
Pi = 3.142
#boltzman const
bc = 1.38e-23
#mass of proton
mi = 1.67e-27
MUK = MU / (8*Pi*Pi*bc)
CON11 = 1.6e-20
CON12 = 9.999999e-21
CON2 = 4.6e-31
UGCONS = 8.310001e3
FRF = 0.3
fe = 1 / 3
FLAG = 0

#Rem reset EINP, energy dissipated by dynamic resistance effect, resistance for non-ohmic materials, generally r = dV/dI. which is 0.5 (Ldot) I^2, generally r =  considering current taking part in the motion

#gamma = specific heat ratio, was just 'g' in original vbs
gamma = 1.66667
#G1 is current sheet speed slower factor
G1 = 2/ (gamma + 1)
G2 = (gamma - 1) / gamma
#Compression ratio of ambient gas swept up by shock front 
GCAP = (gamma + 1) / (gamma - 1)

# Calculate ambient atomic number density (no of atoms per m^2)and some ratios
# For PF operation assume all molecules are completely dissociated in all phases

# N0= number of atoms per m^2; atomic number density
N0 = dissociatenumber * 2.69e25 * P0 / 760
# C is ratio of cathode radius / anode radius
C = RADB / RADA
# Lee paper, page 8. Z0 is the length of anode, so f is ratio of anode length over it's radius
f = Z0 / RADA
# Convert to SI units
C0 = C0 * pow(10,-6)
print(C0)
L0 = L0 * pow(10,-9)
RADB = RADB * 0.01
RADA = RADA * 0.01
Z0 = Z0 * pow(10,-2)
V0 = V0 * 1000
RHO = P0 * 2.33e-4 * MW / 4
TM = 0
#assuming no taper
#Calculate characteristic quantities and scaling parameters
#Lee paper page 5, Voltage at t = 0 over surge impedence
I0 = V0 / numpy.sqrt(L0 / C0)
#Lee paper page 5, 
T0 = numpy.sqrt(L0 * C0)
#Scaling parameter for Axel Phase. Equation 5 on Lee Paper
print(((Z0 * numpy.sqrt(RHO)) / (I0 / RADA)) * (numpy.sqrt(massf) / CURRF))
TA = numpy.sqrt(4 * Pi * Pi * (C * C - 1) / (MU * numpy.log(C))) * ((Z0 * numpy.sqrt(RHO)) / (I0 / RADA)) * (numpy.sqrt(massf) / CURRF)
#Lee papaer page 5, equation 7. zzchar is characteristic axial speed 
ZZCHAR = Z0 / TA
# AL : Alpha, ratio of characteristic electrical discharge time to the characteristic axial transit time
AL = T0 / TA
# Scaling Parameter for Radial Phase. Equation 24a, Lee paper page 8
AA = numpy.sqrt((gamma + 1) * (C * C - 1) / numpy.log(C)) * (f / 2) * (numpy.sqrt(massf / massfr)) * (currfr / CURRF)
# Ratio of stray resistance to surge impedence
RESF = R0 / (numpy.sqrt(L0 / C0))
#Lee paper page 5, equation 9. Ratio between inductance of the axial phase when CS reaches anode end z = z0, over Lo (inductance at t = 0) * current shedding factor, 
BE = 2e-7 * numpy.log(C) * Z0 * CURRF / L0
#Lee paper page 8, equation 24, Scaling parameter for Radial Inward Shock Phase
BF = BE / (numpy.log(C) * f)
#Lee paper page 8, equation 28, VPINCHCH is the speed of radial inward shock
VPINCHCH = ZZCHAR * AA / f
#convert speed of radial inward shock to time
TPINCHCH = RADA / VPINCHCH
#Calculate ratio of characteristic capacitor time to sum of characteristic axial & radial times
ALT = (AL * AA) / (1 + AA)

#Parameters sanity check
print(I0)
if ALT < 0.68:
    raise ValueError("Sorry ALT values are insane")





2.9999999999999997e-05
2.2125699104131396e-11
247716.84715343112


## Axial Phase

In [3]:
#Set time constants
d = 0.002
T = 0
z = 0
ZZ = 0
II = 1
I = 0
IO = 0
AC = AL * numpy.sqrt(1 / 2)





In [10]:
#Integration for Axial Phase
Ipeak = 0
Wpiston = 0
EINP = 0
V = 0
axialphase = []
# Start integration loop
while True:
    T = T + d
    if T > 6:
        raise ValueError("Sorry T is thru the roof!")
    ZZ = ZZ + AC * d
    z = z + ZZ * d
    I = I + II * d
    IO = IO + I * d
    #Init array-of-arrays for axial phase results
   

    #Unit conversion
    TR = T * T0* pow(10,6)
    VR = V * V0*pow(10,-3)
    IR = I * I0*pow(10,-3)
    ZZR = (ZZCHAR / AL) * ZZ * pow(10, -4)
    ACR = ((ZZCHAR / AL) / T0) * AC * pow(10, -10)
    ZR = z * Z0 * 100
    IIR = (II * I0 / T0) * pow(10, -9)
    IOR = IO * I0 * T0

    if IR > Ipeak:
        Ipeak = IR
    #DR is time increments in secs
    DR = d * T0
    #DZR is z increment in m
    DZR = DR * ZZR * pow(10, 4)
    #Integrate to find EINP, energy dissipated by dynamic resistance effect, which is 0.5 (Ldot) I^2, considering current taking part in the motion
    EINP = EINP + pow(10, -7) * (numpy.log(C) * ZZR * pow(10, 4) * IR * IR * pow(10 ,6) * CURRF * CURRF) * DR
    #integrate for piston work
    Wpiston = Wpiston + pow(10, -7) * numpy.log(C) * IR * IR * pow(10, 6) * CURRF * CURRF * DZR
    Lz = pow(10 , -9) * ZR * 2 * numpy.log(C)
    #Einductance is total inductive energy (in all inductances)
    Einductance = 0.5 * pow(10, 6) * IR * IR * (CURRF * CURRF * Lz + L0)
    ni = massf * N0 * ((gamma + 1) / (gamma - 1))
    nimax = N0 * ((gamma + 1) / (gamma - 1))
    #Write out step-by-step integration
    axialstep = []
    axialmap = {}
    axialstep.append(TR)
    axialmap['TR'] = 0
    axialstep.append(IR)
    axialmap['IR'] = 1
    axialstep.append(VR)
    axialmap['VR'] = 2
    axialstep.append(ZR)
    axialmap['ZR'] = 3
    axialstep.append(ZZR)
    axialmap['ZZR'] = 4
    axialstep.append(IR * CURRF)
    axialmap['Plasma I'] = 5
    axialstep.append(Lz * pow(10,-9))
    axialmap['Plasma L'] = 6
    axialstep.append(100 * Einductance / (0.5 * C0 * V0 * V0))
    axialmap['Inductive'] = 7
    axialstep.append(100 * EINP / (0.5 * C0 * V0 * V0))
    axialmap['0.5Ldot'] = 8
    axialstep.append(ni / pow(10,23))
    axialmap['Ni'] = 9
    axialstep.append(TM / pow(10, 6))
    axialmap['T'] = 10
    axialstep.append(nimax / pow(10 , 23))
    axialmap['nimax'] = 11
    axialstep.append(100 * Wpiston / (0.5 * C0 * V0 * V0))
    axialmap['Piston'] = 12
    axialphase.append(axialstep)
    V = z * II + ZZ * I
    V = V * BE
    #Compute Generating Quantities (ie acceleration and IDOT) before loopback to continue integration
    AC = (AL * AL * I * I - ZZ * ZZ) / z
    II = (1 - IO - BE * ZZ * I - RESF * I) / (1 + BE * z)
    #Check end condition reached
    if z >= 1:
        break

#prep for next phase
Laxial = ZR * 2 * numpy.log(C)
ZG = ZZ
CFA = CURRF
#Introduce differential in current factors for axial and radial phases
CurrentFactorRatio = currfr / CURRF
CFR = CurrentFactorRatio
BE = BE * CFR
BF = BF * CFR
CURRF = CURRF * CFR
AAg = AA / numpy.sqrt(gamma + 1)
#End of Axial Phase
#print(axialphase[0])

# Radial Inward Shock Phase
Radial phase _RI_, distances are relative to radius _a_.  
_KS_ is shock position, _KP_ is radial piston position, _ZF_ is focus  
pinch length, all normalized to inner radius _a_; _VS_ and _VP_ are  
radial shock and piston speed,_VZ_ is axial pinch length elongating rate  
Distances, radius and speeds are relative to radius of anode.  
AS BEFORE QUANTITIES WITH AN _R_ ATTACHED HAVE BEEN RE-COMPUTED AS REAL, i.e. UN-NORMALIZED QUANTITIES EXPRESSED IN USUAL LABORATORY UNITS.  
FOR THIS PHASE _Z_=EFFECTIVE CHARGE NUMBER!!!

In [8]:
#Start of Radial Inward Shock Phase


#init the shock array
radialshock = []
#Here is where we deviate from original vbs code. Since we're using vectors, no need to append to Axial phase
rowi = 0
FIRSTRADIALROW = rowi

#Set some initial values for Radial Inward Phase step-by-step integration
#Reset time increment to finer step-size

KS = 1
KP = 1 
ZF = math.exp(-5)
zFLAG = 0 
gFLAG = 0
#SET TIME INCREMENT TO HAVE ABOUT 1500 (up to 3000 for high pressure) STEPS IN RADIAL INWARD SHOCK PHASE
DREAL = TPINCHCH / 700
d = DREAL / T0
#set initial
IDELAY = I 
KPDELAY = KP
KSDELAY = KS
VSDELAY = -1
# Set initial value, approximately, for CHARGE NUMBER Z
# For H2,D2 and He, assume fully ionized with gamma=1.667 during all of radial phase
z = ZN
# Start Step-by-step integration of Radial Inward Shock Phase, in non-dimensional units
# First, compute Inward shock speed
trradialstart = T * T0
# Need to display trradialstart expressed in 10^6
GCAP = (gamma + 1) / (gamma - 1)
AA = AAg * numpy.sqrt(gamma + 1)
VS = -AL * AA * IDELAY / (KPDELAY)
VSR = VS * RADA / T0
# Real temperature is needed to DETERMINE SMALL DISTURBANCE SPEED FOR COMMUNICATION TIME CORRECTION.
# Hence the shock speed is re-calculated in SI units, then Plasma Temp TM is calculated, based on shock theory
TM = (MW / (8315)) * ((GCAP - 1) / (GCAP * GCAP)) * ((VSR * VSR) / ((1 + z) * dissociatenumber))
TeV = TM / (1.16 * pow(10,4))
#Select Table for G & Z; according to gas. But only handling z = 1 for now
G1 = 2 / (gamma + 1)
G2 = (gamma - 1) / gamma
if FLAG == 10:
    False
else:
    True

VZ = -G1 * VS
K1 = KS / KP
E1 = G1 * K1 * VSDELAY
E2 = (1 / gamma) * (KP / I) * (1 - K1 * K1) * II
E3 = (G1 / 2) * (KP / ZF) * (1 - K1 * K1) * VZ
E4 = G2 + (1 / gamma) * K1 * K1
VP = (E1 - E2 - E3) / E4
V = (BE - BF * (numpy.log(KP / C)) * ZF) * II - I * (BF * (ZF / KP) * VP + (BF * (numpy.log(KP / C))) * VZ)
II = (1 - IO + BF * I * (ZF / KP) * VP + BF * (numpy.log(KP / C)) * I * VZ - RESF * I) / (1 + BE - BF * (numpy.log(KP / C)) * ZF)
# Increment time and Integrate, by linear approx, for I, flowed charge I0, KS, KP and ZF
T = T + d
I = I + II * d
IO = IO + I * d
KS = KS + VS * d
KP = KP + VP * d
ZF = ZF + VZ * d
#Rem * Re-scales speeds, distances and time to real, convenient units
SSR = VS * (RADA / T0) * pow(10, -4)
SPR = VP * (RADA / T0) * pow(10, -4)
SZR = VZ * (RADA / T0) * pow(10, -4)
KSR = KS * RADA * 1000
kpr = KP * RADA * 1000
zfr = ZF * RADA * 1000

TR = T * T0 * pow(10,6)
VR = V * V0 * pow(10, -3)
IR = I * I0 * pow(10, -3)
IIR = (II * I0 / T0) * pow(10,-9)
# DR is time increment in secs, DKPR is piston position increment & DZFR length position increment, both in SI units    
DR = d * T0
DKPR = SPR * DR * pow(10, 4)
DZFR = SZR * DR * pow(10, 4)
if IR > Ipeak: 
    Ipeak = IR

#Obtain Max induced voltage
if VR > VRMAX:
    VRMAX = VR

TRRadial = TR * pow(10,3) - trradialstart * pow(10,9)

if SSR < peakvs:
    peakvs = SSR
if SPR < peakvp: 
    peakvp = SPR

#Integrate to find EINP, energy dissipated by dynamic resistance effect, which is 0.5 (Ldot) I^2, considering current taking part in the motion
EINP = EINP + pow(10,3) * (SZR * numpy.log(1000 * RADB / kpr) - (SPR * (zfr / kpr))) * IR * IR * CURRF * CURRF * DR
# Also integrate for piston work
Wpiston = Wpiston + 0.1 * (DZFR * numpy.log(1000 * RADB / kpr) - DKPR * (zfr / kpr)) * IR * IR * CURRF * CURRF
Lplasma = Laxial + (zfr * 2 * (10 ^ -1) * numpy.log(1000 * RADB / kpr))
Einductance = 0.5 * pow(10,6) * IR * IR * (CURRF * CURRF * Lplasma * pow(10,-9) + L0)
ni = massfr * N0 * ((gamma + 1) / (gamma - 1))
nimax = N0 * ((gamma + 1) / (gamma - 1))
step = []

step.append(TR)
step.append(IR)
step.append(VR)
step.append(TR)
step.append(IR * CURRF)
step.append(KSR)
step.append(kpr)
step.append(zfr)
step.append(SSR)
step.append(SPR)
step.append(SZR)
step.append(Lplasma)
step.append(100 * Einductance / (0.5 * C0 * V0 * V0))
step.append(100 * EINP / (0.5 * C0 * V0 * V0))
step.append(ni / pow(10,23))
step.append(TM / pow(10,6))
step.append(nimax / pow(10,3))
step.append(100 * Wpiston / (0.5 * C0 * V0 * V0))

#Testing
step.append(SZR)
step.append(DR)
step.append(DZFR)
step.append(-SPR*DR)
step.append(DKPR)
step.append(TR)
step.append(TRRadial)
step.append(IR)
step.append(VR)
step.append(KSR)
step.append(kpr)
step.append(zfr)
step.append(SSR)
step.append(SPR)
step.append(SZR)
step.append(TM)
step.append(gamma)
step.append(z)
step.append(EINP)

radialshock.append(step)
rowi = rowi+1
#no taper

#To apply finite small disturbance speed correction. Compute propagation time and the 'lookback' row number
if KSR <= kpr:
    SDSPEED = numpy.sqrt(gamma * dissociatenumber * (1 + z) * bc * TM / (MW * mi))
    SDDELAYTIME = ((kpr - KSR) / 1000) / SDSPEED
    backhowmanyrows = SDDELAYTIME / DR
    BACKROWNUMBER = rowi - backhowmanyrows
    if BACKROWNUMBER < FIRSTRADIALROW: 
            BACKROWNUMBER = FIRSTRADIALROW + 1
    delayrow = BACKROWNUMBER - 1
    if delayrow < 0:
        delayrow = 0
    #Rem Look back to appropriate row to obtain 'lookback' quantities; also non-dimensionalize these quantities
    

